In [1]:
import yfinance as yf
import datetime
import numpy as np
from ta import momentum, trend
import pandas as pd
import plotly.graph_objects as go
def Get_Main_Data(symbol):
    symbol = "^NSEI"
    # HDFCBANK.NS
    ticker = yf.Ticker(symbol)
    end_date = datetime.date.today()
    df = ticker.history(period="80d", interval="1d")
    
    df.reset_index(inplace=True)
    df['Date'] = df['Date'].dt.date
    
    df['5EMA'] = trend.EMAIndicator(df['Close'], window=10).ema_indicator()
    
    # Create boolean conditions for uptrend and downtrend
    uptrend_condition = df['5EMA'] > df['5EMA'].shift(1)
    downtrend_condition = df['5EMA'] < df['5EMA'].shift(1)
    
    # Count consecutive True values within a rolling window
    consecutive_uptrends = uptrend_condition.rolling(window=3).sum()
    consecutive_downtrends = downtrend_condition.rolling(window=3).sum()
    
    # Use the consecutive counts to determine trend
    df['Day_Trend'] = np.where(consecutive_uptrends >= 2, 'Uptrend', np.where(consecutive_downtrends >= 2, 'Downtrend', 'Sideways'))
    
    df['Day_Candle'] = np.where(df['Close'] > df['Open'], 'Green', 'Red')
    
    for i in range(1, 3):
        df[f'{i}Prev_Day_Close'] = df['Close'].shift(i)
        df[f'{i}Prev_Day_High'] = df['High'].shift(i)
        df[f'{i}Prev_Day_Low'] = df['Low'].shift(i)
        df[f'{i}Prev_Day_Candle'] = df['Day_Candle'].shift(i)
        
    df['Day_Open'] =  df['Open'] 
    df['Day_High'] =  df['High']
    df['Day_Low'] =  df['Low']
    df['Day_Close'] =  df['Close']

    df['Market_Opening_Gap'] = df['Day_Open'] - df['1Prev_Day_Close']
    
    columns_to_drop = ['Volume', 'Dividends', 'Stock Splits', '5EMA', 'Low', 'Close', 'High', 'Open']
    df.drop(columns=columns_to_drop, inplace=True)
    
    df = df.dropna()
    nifty_50_df = yf.download(symbol, period="5d", interval="5m")
    nifty_50_df.reset_index(inplace=True)
    nifty_50_df['Date'] = nifty_50_df['Datetime'].dt.date
    nifty_50_df['Time'] = nifty_50_df['Datetime'].dt.time

    # --------------------------
    
    nifty_50_df['5EMA'] = trend.EMAIndicator(nifty_50_df['Close'], window=30).ema_indicator()
    
    uptrend_condition_2 = nifty_50_df['5EMA'] > nifty_50_df['5EMA'].shift(1)
    downtrend_condition_2 = nifty_50_df['5EMA'] < nifty_50_df['5EMA'].shift(1)
    
    consecutive_uptrends_2 = uptrend_condition_2.rolling(window=4).sum()
    consecutive_downtrends_2 = downtrend_condition_2.rolling(window=4).sum()
    
    # Determine trend based on conditions
    nifty_50_df['Trend'] = np.where((consecutive_uptrends_2 > 0) & (consecutive_downtrends_2 == 0), 'Uptrend',
                                    np.where((consecutive_downtrends_2 > 0) & (consecutive_uptrends_2 == 0), 'Downtrend', 'Sideways'))



    nifty_50_df['Candle'] = np.where(nifty_50_df['Close'] > nifty_50_df['Open'], 'Green', 'Red')
    nifty_50_df['RSI'] = round(momentum.RSIIndicator(nifty_50_df['Close'], window=4).rsi(),2)
    
    del nifty_50_df['Datetime']
    del nifty_50_df['Adj Close']
    del nifty_50_df['Volume']
    
    # Convert the 'Date' column in df to a common format
    df['Date'] = pd.to_datetime(df['Date'])
    nifty_50_df['Date'] = pd.to_datetime(nifty_50_df['Date'])
    merged_df = df.merge(nifty_50_df, on='Date', how='inner')
    desired_columns = [
            'Date',
            'Time',
            'Market_Opening_Gap',
            'Open',
            'High',
            'Low',
            'Close',
            'Candle',
            'RSI',
            'Trend',
            'Day_Open',
            'Day_High',
            'Day_Low',
            'Day_Close',
            'Day_Trend',
            'Day_Candle',
            '1Prev_Day_Close',
            '1Prev_Day_High',
            '1Prev_Day_Low',
            '1Prev_Day_Candle',
            '2Prev_Day_Close',
            '2Prev_Day_High',
            '2Prev_Day_Low',
            '2Prev_Day_Candle'
        ]

    merged_df = merged_df[desired_columns]

    return round(merged_df,2)
data  = Get_Main_Data("^NSEI")

[*********************100%%**********************]  1 of 1 completed


In [2]:
def find_pattern_all(df):
    df["upper_shadow"] = df['High'] - df[['Open', 'Close']].max(axis=1)
    df["lower_shadow"] = df[['Open', 'Close']].min(axis=1) - df['Low']
    df["body"] = abs(df["Close"] - df["Open"])
    df["full"] = df['High'] - df['Low']
    df["Candle"] = df["Close"].where(df["Close"] >= df["Open"], "Red")

    df['Prev_Open'] = df['Open'].shift(1)
    df['Prev_Close'] = df['Close'].shift(1)
    df['Prev_High'] = df['High'].shift(1)
    df['Prev_Low'] = df['Low'].shift(1)
    df['Prev_Candle'] = df['Candle'].shift(1)

    df['Next_Open'] = df['Open'].shift(-1)
    df['Next_Close'] = df['Close'].shift(-1)
    df['Next_High'] = df['High'].shift(-1)
    df['Next_Low'] = df['Low'].shift(-1)
    df['Next_Candle'] = df['Candle'].shift(-1)

    n = 5  # common variable for doji, dragonfly doji, gravestone doji

    df['Engulfing_Bullish'] = (df["Prev_Candle"] == "Red") & (df["Open"] <= df["Prev_Close"]) & (df["Close"] >= df["Prev_Open"]) & (df['Next_Candle'] == "Green")
    df['Engulfing_Bearish'] = (df["Prev_Candle"] == "Green") & (df["Open"] >= df["Prev_Close"]) & (df["Close"] <= df["Prev_Open"]) & (df['Next_Candle'] == "Red")
    df['Doji'] = (df["body"] < df['full'] / n) & (df['upper_shadow'] >= n * df["body"]) & (df['lower_shadow'] >= n * df["body"])
    df['Dragonfly_Doji_Bullish'] = (df["body"] < df['full'] / n) & (df['upper_shadow'] * 2 < df['lower_shadow'])
    df['Gravestone_Doji_Bearish'] = (df["body"] < df['full'] / n) & (df['upper_shadow'] > df['lower_shadow'] * 2)
    df['Evening_Bearish'] = (df["Prev_Candle"] == "Green") & (df["Next_Candle"] == "Red") & (abs(df["Prev_Close"] - df["Prev_Open"]) > df["body"])
    df['Morning_Bullish'] = (df["Prev_Candle"] == "Red") & (df["Next_Candle"] == "Green") & (abs(df["Prev_Close"] - df["Prev_Open"]) > df["body"])
    df['Hammer_Bullish'] = df['upper_shadow'] * 2 < df['lower_shadow']
    df['Shooting_Star_Bearish'] = df['upper_shadow'] > df['lower_shadow'] * 2
    df['Harami_Bearish'] = (df["Prev_Candle"] == "Green") & (abs(df["Prev_Close"] - df["Prev_Open"]) > df["body"] * 2) & (df["High"] < df["Prev_High"]) & (df["Close"] < df["Prev_Close"])
    df['Harami_Bullish'] = (df["Prev_Candle"] == "Red") & (abs(df["Prev_Close"] - df["Prev_Open"]) > df["body"] * 2) & (df["High"] < df["Prev_High"]) & (df["Close"] < df["Prev_Close"])

    bullish_columns = [col for col in df.columns if 'Bullish' in col and df[col].eq(True).any()]
    bearish_columns = [col for col in df.columns if 'Bearish' in col and df[col].eq(True).any()]

    df['Bullish_Count'] = df[bullish_columns].eq(True).sum(axis=1)
    df['Bearish_Count'] = df[bearish_columns].eq(True).sum(axis=1)

    df['Pattern_Name'] = df[bullish_columns + bearish_columns].apply(lambda x: x.index[x].tolist(), axis=1)
    
    delete_columns = ['upper_shadow', 'lower_shadow', 'body', 'full', 'Prev_Open', 'Prev_Close', 'Prev_High', 'Prev_Low', 'Prev_Candle', 'Next_Open', 'Next_Close', 'Next_High', 'Next_Low', 'Next_Candle', 'Engulfing_Bullish', 'Engulfing_Bearish', 'Doji', 'Dragonfly_Doji_Bullish', 'Gravestone_Doji_Bearish', 'Evening_Bearish', 'Morning_Bullish', 'Hammer_Bullish', 'Shooting_Star_Bearish', 'Harami_Bearish', 'Harami_Bullish']
    df.drop(columns = delete_columns, inplace=True)
    return df

data = find_pattern_all(data)
data

,Date,Time,Market_Opening_Gap,Open,High,Low,Close,Candle,RSI,Trend,...,1Prev_Day_High,1Prev_Day_Low,1Prev_Day_Candle,2Prev_Day_Close,2Prev_Day_High,2Prev_Day_Low,2Prev_Day_Candle,Bullish_Count,Bearish_Count,Pattern_Name
0,2023-11-03,09:15:00,107.75,19241.00,19257.75,19235.10,19247.80,19247.8,NaN,Sideways,...,19175.25,19064.15,Green,18989.15,19096.05,18973.7,Red,0,0,[]
1,2023-11-03,09:20:00,107.75,19248.35,19255.05,19243.20,19246.15,Red,NaN,Sideways,...,19175.25,19064.15,Green,18989.15,19096.05,18973.7,Red,0,2,"[Gravestone_Doji_Bearish, Shooting_Star_Bearish]"
2,2023-11-03,09:25:00,107.75,19245.95,19253.80,19242.20,19248.55,19248.55,NaN,Sideways,...,19175.25,19064.15,Green,18989.15,19096.05,18973.7,Red,0,0,[]
3,2023-11-03,09:30:00,107.75,19248.60,19264.90,19248.45,19249.70,19249.7,76.06,Sideways,...,19175.25,19064.15,Green,18989.15,19096.05,18973.7,Red,0,2,"[Gravestone_Doji_Bearish, Shooting_Star_Bearish]"
4,2023-11-03,09:35:00,107.75,19249.75,19262.10,19244.55,19260.45,19260.45,94.90,Sideways,...,19175.25,19064.15,Green,18989.15,19096.05,18973.7,Red,1,0,[Hammer_Bullish]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,2023-11-09,15:05:00,13.90,19394.50,19400.45,19388.55,19393.75,Red,27.94,Downtrend,...,19464.40,19401.50,Red,19406.70,19423.50,19329.1,Green,0,0,[]
371,2023-11-09,15:10:00,13.90,19392.95,19399.40,19389.55,19393.10,19393.1,25.95,Downtrend,...,19464.40,19401.50,Red,19406.70,19423.50,19329.1,Green,1,0,[Harami_Bullish]
372,2023-11-09,15:15:00,13.90,19392.55,19400.60,19389.75,19398.45,19398.45,58.42,Downtrend,...,19464.40,19401.50,Red,19406.70,19423.50,19329.1,Green,0,0,[]
373,2023-11-09,15:20:00,13.90,19398.10,19403.35,19391.60,19400.45,19400.45,65.88,Downtrend,...,19464.40,19401.50,Red,19406.70,19423.50,19329.1,Green,1,0,[Hammer_Bullish]
